In [21]:

# Import necessary libraries
import cv2
import numpy as np
from tensorflow.keras.models import model_from_json
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from ipywidgets import Button, Output
import ipywidgets as widgets
import json


In [22]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# !pip install tensorflow opencv-python-headless ipywidgets
# !pip install --upgrade ipywidgets

In [24]:
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.15.0


In [25]:
# Load the model
with open('/content/drive/MyDrive/Colab Notebooks/Facial Emotion Recognition/model.json', 'r') as json_file:
    loaded_model_json = json_file.read()

model_config = json.loads(loaded_model_json)

for layer in model_config['config']['layers']:
    if layer['class_name'] == 'InputLayer':
        if 'batch_shape' in layer['config']:
            layer['config']['batch_input_shape'] = layer['config'].pop('batch_shape')

modified_model_json = json.dumps(model_config)

model = tf.keras.models.model_from_json(modified_model_json)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.load_weights("/content/drive/MyDrive/Colab Notebooks/Facial Emotion Recognition/model.weights.h5")

print("Model loaded successfully")


Model loaded successfully


In [26]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 46, 46, 64)        640       
                                                                 
 conv2d_3 (Conv2D)           (None, 46, 46, 64)        36928     
                                                                 
 batch_normalization_1 (Bat  (None, 46, 46, 64)        256       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 23, 23, 64)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 23, 23, 128)      

In [27]:
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [28]:
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = 'Capture';
        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
        }
        ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

In [33]:

def process_image(image_path):
    # Read the image
    frame = cv2.imread(image_path)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        # Extract the face ROI
        roi_gray = gray[y:y+h, x:x+w]

        # Preprocess the ROI
        roi = cv2.resize(roi_gray, (48, 48))
        roi = roi.astype("float") / 255.0
        roi = np.expand_dims(roi, axis=0)
        roi = np.expand_dims(roi, axis=-1)

        # Make a prediction
        preds = model.predict(roi)[0]
        emotion_idx = np.argmax(preds)
        emotion = emotion_labels[emotion_idx]

        # Draw rectangle around face and label with predicted emotion
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return frame

# Create a button widget
button = widgets.Button(description="Take a photo")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        output.clear_output()
        filename = take_photo()
        print("Photo taken!")

        # Process the image
        result_image = process_image(filename)

        # Display the result
        cv2_imshow(result_image)

button.on_click(on_button_clicked)

Button(description='Take a photo', style=ButtonStyle())

Output()